In [ ]:
import math as m
import itertools as its

#Preliminary functions needed for computation

def CreateBlockMatrix(dimensions, blockshape, blocklist):
    '''Create one matrix out of several specified blocks'''
    return block_matrix(dimensions[0],dimensions[1], [blocklist[t] for t in blockshape])



def is_zeroone(matrix):
    '''Check if a matrix is a zero-one matrix'''
    for row in matrix:
        for entry in row:
            if entry not in {0,1}:
                return False

    return True



def switchingblock(level, blocknumber, sort):
    '''Create a regular orthogonal permutation-based matrix of a certain level, number of blocks and sort'''
    J = ones_matrix(level)
    O = zero_matrix(level)
    Y = level*identity_matrix(level)- J

    def rotate(l):
        return [l[-1]] + l[:-1]

    baserow = [0,2] + [1]*(blocknumber-2)
    shapematrix = []
    for _ in range(blocknumber):
        shapematrix = shapematrix + baserow
        baserow = rotate(baserow)


    if sort == 'wqh':
        return CreateBlockMatrix([blocknumber,blocknumber], shapematrix, [Y,O,J])/level
    if sort == 'gm':
        return CreateBlockMatrix([blocknumber,blocknumber], shapematrix, [-Y,O,J])/level


def orbitQinf(n, blocks, level, sort, recursionlevel):
    '''This function recursively finds all permutations (as matrices) that do not fix the switching matrix Q of the infinite family.
    At each recursion level it takes any set of size equal to the level (containing numbers that we not put to the front before) and puts it at the front of the permutation. 
    The matrix Q has a symmetry group induced by rotating the blocks and any permutation within the blocks. 
    Therefore it only matters what the set of vertices in each block are, so we pick combinations 
    and by picking from [1,....,blocks*level-1] we ensure that the last block contains the highest number, so we deal with the symmetry of the block rotation.'''

    #in case of GM the orthogonal matrix is symmetric.
    if blocks == 2 and sort == 'gm':
        return [identity_matrix(n)]
    if recursionlevel == 1:
        return [identity_matrix(n)]

    def getvertices(n, S):
        #pulls vertices in S to the front of the list.
        complement = set(range(1,n+1)) - set(S)
        return S + list(complement)

    matrices= []
    for X in Combinations(range(level*(blocks-recursionlevel)+1, blocks*level), level):
        Xmatrix = Permutation(getvertices(n, X)).to_matrix()
        for P in orbitQinf(n, blocks, level, sort, recursionlevel-1):
            # As the matrices will be transposed (on the left) in bruteswitching. 
            # The way to ensure that the highest level does the second to last block is to multiply Xmatrix on the left. 
            matrices.append(Xmatrix*P)

    return matrices


def Genswitchingset(level, blocks, sort):
    '''Create all labelled graphs (up to complementation) that form a switching set for the Q(level,blocks, sort)-switching method
        up to permutation that leaves Q fixed.'''
    switchingset = []

    Q = switchingblock(level, blocks, sort)
    Permlist = orbitQinf(blocks*level, blocks, level, sort, blocks)
    n = level*blocks
    for G in graphs.nauty_geng(str(n) + ' 0:' + str(m.floor(n*(n-1)/4))):
        A = G.adjacency_matrix()
        for P in Permlist:
            B = Q.transpose()*P.transpose()*A*P*Q
            if is_zeroone(B):
                switchingset.append(P.transpose()*A*P)
                break

    return [k for k, _ in its.groupby(sorted(switchingset))]

def Qresp(Q,m):
    '''Find all Q-respecting vectors for any Q of size m'''
    qrespvectors = []
    for vec in its.product(range(2), repeat = m):
        if all(i in {0,1} for i in vector(vec)*Q):
            qrespvectors.append(vector(vec))
    
    return qrespvectors

def pad_with_identity(P, size):
    '''Pad matrix P with an identity matrix to make it a square matrix of given size'''
    I = identity_matrix(size)
    for i in range(P.nrows()):
        for j in range(P.ncols()):
            I[i, j] = P[i, j]
    return I

#Compute Aut_Q(Gamma) for each switching graph of GM8 (or WQH8 if you change the sort to 'WQH')

def sumAutQ(level, blocks, sort):
    '''Compute the sum of 1/|Aut_Q(Gamma)| for each switching graph of the Q-switching method'''
    switchingset = Genswitchingset(level, blocks, sort)
    Q = switchingblock(level, blocks, sort)
    qrespvectors = Qresp(Q,level*blocks)
    qrespset = {tuple(v) for v in qrespvectors}

    graphlist = []

    for matrix in switchingset:
        G =Graph(matrix)
        graphlist += [G,G.complement()]
        
    autqgammalist = []
    for G in graphlist:
        autqgamma = []
        aut =  G.automorphism_group()
        for perm in aut:
            P = perm.matrix()
            P_padded = pad_with_identity(P, level*blocks)
            if all(tuple(P_padded.transpose()*v) in qrespset for v in qrespvectors):
                autqgamma.append(P_padded)
        autqgammalist.append(autqgamma)
    return sum([1/len(aut) for aut in autqgammalist]), checkisomorphism(graphlist)

def checkisomorphism(graphlist):
    '''Check if there are any isomorphic graphs in the list'''
    for G in graphlist:
        checkiso = 0
        for H in graphlist:
            if G.is_isomorphic(H):
                checkiso += 1
        #all graphs are isomorphic to themselves but this will show that there are no other isomorphic graphs
        if checkiso > 1:
            return G, 'isomorphic'
    return None

print(sumAutQ(4,2, 'gm'))
print(sumAutQ(4,2, 'wqh'))



(319/280, None)
(20213/576, None)
